In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from ipykernel.kernelapp import kernel_aliases
from pySmartDL.download import download
from tensorflow.python.ops.gen_dataset_ops import padded_batch_dataset
from torchvision import datasets, transforms
from torch.utils.data import DataLoader


In [2]:
device= "cuda" if torch.cuda.is_available() else "cpu"


In [4]:
#Hyperparameter

input_size=28
hidden_size=256
num_layers=2
sequence_len=28
learning_rate=0.001
batch_size=64
num_classes=10




In [6]:
# Load data set
train =datasets.MNIST("",train=True,
                      transform= transforms.Compose([transforms.ToTensor()]))

test =datasets.MNIST("",train=False,
                      transform= transforms.Compose([transforms.ToTensor()]))

train_loader = DataLoader(dataset=train, batch_size=batch_size, shuffle=True)
test_loader  = DataLoader(dataset=test,  batch_size=batch_size, shuffle=False)

In [17]:
import torch
import torch.nn as nn

class BRNN(nn.Module):  # 👈 use nn.Module not nn.modules
    def __init__(self, input_size, hidden_size, num_layers, num_classes):
        super().__init__()

        self.hidden_size = hidden_size
        self.num_layers = num_layers

        self.lstm = nn.LSTM(
            input_size, hidden_size, num_layers,
            batch_first=True, bidirectional=True # IN BIDIRICTIONAL THE NO OF NEURONS IE THE hidden size become 2 times to make forward and backward pass
        )

        self.fc = nn.Linear(hidden_size * 2, num_classes)

    def forward(self, x):
        h0 = torch.zeros(self.num_layers * 2, x.size(0), self.hidden_size).to(device)
        c0 = torch.zeros(self.num_layers * 2, x.size(0), self.hidden_size).to(device)

        out, _ = self.lstm(x, (h0, c0))

        out = self.fc(out[:, -1, :])
        return out


In [22]:
num_epochs=3
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        # Move to device
        images = images.reshape(-1, sequence_len, input_size).to(device)  # [batch, 28, 28]
        labels = labels.to(device)

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backprop
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if (i+1) % 100 == 0:
            print(f"Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{len(train_loader)}], Loss: {loss.item():.4f}")


Epoch [1/3], Step [100/938], Loss: 0.6803
Epoch [1/3], Step [200/938], Loss: 0.6494
Epoch [1/3], Step [300/938], Loss: 0.3410
Epoch [1/3], Step [400/938], Loss: 0.2161
Epoch [1/3], Step [500/938], Loss: 0.1565
Epoch [1/3], Step [600/938], Loss: 0.1953
Epoch [1/3], Step [700/938], Loss: 0.2101
Epoch [1/3], Step [800/938], Loss: 0.0963
Epoch [1/3], Step [900/938], Loss: 0.0369
Epoch [2/3], Step [100/938], Loss: 0.2097
Epoch [2/3], Step [200/938], Loss: 0.1329
Epoch [2/3], Step [300/938], Loss: 0.1413
Epoch [2/3], Step [400/938], Loss: 0.0869
Epoch [2/3], Step [500/938], Loss: 0.1427
Epoch [2/3], Step [600/938], Loss: 0.0738
Epoch [2/3], Step [700/938], Loss: 0.1190
Epoch [2/3], Step [800/938], Loss: 0.0982
Epoch [2/3], Step [900/938], Loss: 0.2791
Epoch [3/3], Step [100/938], Loss: 0.0112
Epoch [3/3], Step [200/938], Loss: 0.0168
Epoch [3/3], Step [300/938], Loss: 0.0409
Epoch [3/3], Step [400/938], Loss: 0.0500
Epoch [3/3], Step [500/938], Loss: 0.0880
Epoch [3/3], Step [600/938], Loss:

In [21]:
model = BRNN(input_size, hidden_size, num_layers, num_classes).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

In [24]:
# Testing
model.eval()
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.reshape(-1, sequence_len, input_size).to(device)
        labels = labels.to(device)
        outputs = model(images)

        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print(f"Test Accuracy: {100 * correct / total:.2f}%")

Test Accuracy: 98.35%
